In [1]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

In [2]:
def get_marked_centers(image_path, target_color=(255, 0, 0)):
    """Finds manually marked center points by detecting a specific color in the image."""
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for the target color (tuned for pure red marks)
    lower_bound = np.array([0, 100, 100])
    upper_bound = np.array([10, 255, 255])
    
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # Find the nonzero points (locations of the marked centers)
    points = cv2.findNonZero(mask)
    
    centers = []
    if points is not None:
        for point in points:
            x, y = point[0]
            centers.append((int(x), int(y)))
    
    return centers

def get_text_color(background_color: tuple[int, int, int, int]):
    """Returns black or white text color based on the brightness of the background."""
    r, g, b = background_color[:3]
    brightness = (r * 299 + g * 587 + b * 114) / 1000
    return (0,0,0) if brightness > 128 else (189, 199, 217)

def color_territories(image_path, territory_colors, territory_armies, output_path):
    """Colors the given territories on the RISK map and overlays army counts with contrast-aware text."""
    image = Image.open(image_path).convert("RGBA")
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default(size=40)
    
    for coords, color in territory_colors.items():
        ImageDraw.floodfill(image, coords, color, thresh=30)
    
    # Overlay army counts
    for (x, y), armies in territory_armies.items():
        text = str(armies)
        text_size = draw.textbbox((0, 0), text, font=font)
        text_x, text_y = text_size[2] - text_size[0], text_size[3] - text_size[1]
        
        adjusted_x = x - text_x // 2
        adjusted_y = y - text_y // 2
        
        background_color = image.getpixel((x, y))
        text_color = get_text_color(background_color)
        
        draw.text((adjusted_x, adjusted_y), text, fill=text_color, font=font)
    
    image.save(output_path)
    print(f"Image saved to {output_path}")

In [3]:
import pathlib
import random

marked_image_path = pathlib.Path("data/risk_board_2.png")
clear_image_path = pathlib.Path("data/risk_board.png")
output_path = pathlib.Path("data/colored_map.png")

centers = get_marked_centers(marked_image_path, (191,64,64)) or [(224,324), (484, 330), (462, 488), (654, 520), (856, 516), (476, 706), (692, 756)]
print(f"Detected {len(centers)} centers: {centers}")

territory_colors = {center: (random.randint(0, 255),random.randint(0, 255),random.randint(0, 255), 255) for center in centers}

territory_armies = {center: ind for ind, center in enumerate(centers)}

color_territories(clear_image_path, territory_colors, territory_armies, output_path)

Detected 42 centers: [(1022, 204), (2420, 273), (2645, 277), (443, 310), (225, 322), (2198, 348), (1270, 429), (1470, 445), (466, 480), (2047, 489), (842, 520), (640, 526), (2363, 549), (1745, 569), (1264, 685), (1481, 691), (456, 701), (2398, 704), (2716, 744), (1993, 761), (662, 780), (1494, 843), (474, 891), (2358, 898), (1260, 950), (1833, 996), (2177, 1068), (708, 1121), (2406, 1142), (1582, 1154), (1352, 1243), (912, 1300), (1722, 1328), (755, 1385), (2715, 1396), (2464, 1442), (1599, 1493), (772, 1594), (2749, 1647), (2581, 1751), (1639, 1759), (1892, 1770)]
Image saved to data/colored_map.png


In [4]:
coords = {
    "GREENLAND": (1022, 204),
    "YAKUTSK": (2420, 273),
    "KAMCHATKA": (2645, 277),
    "NORTHWEST_TERRITORY": (443, 310),
    "ALASKA": (225, 322),
    "SIBERIA": (2198, 348),
    "ICELAND": (1270, 429),
    "SCANDINAVIA": (1470, 445),
    "ALBERTA": (466, 480),
    "URAL": (2047, 489),
    "QUEBEC": (842, 520),
    "ONTARIO": (640, 526),
    "IRKUTSK": (2363, 549),
    "RUSSIA": (1745, 569),
    "GREAT_BRITAIN": (1264, 685),
    "NORTHERN_EUROPE": (1481, 691),
    "WESTERN_UNITED_STATES": (456, 701),
    "MONGOLIA": (2398, 704),
    "JAPAN": (2716, 744),
    "AFGHANISTAN": (1993, 761),
    "EASTERN_UNITED_STATES": (662, 780),
    "SOUTHERN_EUROPE": (1494, 843),
    "CENTRAL_AMERICA": (474, 891),
    "CHINA": (2358, 898),
    "WESTERN_EUROPE": (1260, 950),
    "MIDDLE_EAST": (1833, 996),
    "INDIA": (2177, 1068),
    "VENEZUELA": (708, 1121),
    "SIAM": (2406, 1142),
    "EGYPT": (1582, 1154),
    "NORTH_AFRICA": (1352, 1243),
    "BRAZIL": (912, 1300),
    "EAST_AFRICA": (1722, 1328),
    "PERU": (755, 1385),
    "NEW_GUINEA": (2715, 1396),
    "INDONESIA": (2464, 1442),
    "CONGO": (1599, 1493),
    "ARGENTINA": (772, 1594),
    "EASTERN_AUSTRALIA": (2749, 1647),
    "WESTERN_AUSTRALIA": (2581, 1751),
    "SOUTH_AFRICA": (1639, 1759),
    "MADAGASCAR": (1892, 1770),
}

territory_armies = {(225, 322): '1', (466, 480): '1', (474, 891): '1', (662, 780): '1', (1022, 204): '?', (443, 310): '?', (640, 526): '1', (842, 520): '?', (456, 701): '1', (772, 1594): '?', (912, 1300): '1', (755, 1385): '?', (708, 1121): '?', (1264, 685): '?', (1270, 429): '?', (1481, 691): '1', (1745, 569): '1', (1470, 445): '1', (1494, 843): '?', (1260, 950): '1', (1599, 1493): '?', (1722, 1328): '?', (1582, 1154): '?', (1892, 1770): '1', (1352, 1243): '1', (1639, 1759): '?', (1993, 761): '1', (2358, 898): '?', (2177, 1068): '?', (2363, 549): '?', (2716, 744): '1', (2645, 277): '1', (1833, 996): '?', (2398, 704): '?', (2406, 1142): '?', (2198, 348): '1', (2047, 489): '1', (2420, 273): '?', (2749, 1647): '1', (2464, 1442): '?', (2715, 1396): '1', (2581, 1751): '1'}
territory_colors = {(225, 322): (52, 194, 95), (466, 480): (52, 194, 95), (474, 891): (52, 194, 95), (662, 780): (52, 194, 95), (1022, 204): (128, 128, 128), (443, 310): (128, 128, 128), (640, 526): (52, 194, 95), (842, 520): (128, 128, 128), (456, 701): (52, 194, 95), (772, 1594): (128, 128, 128), (912, 1300): (52, 194, 95), (755, 1385): (128, 128, 128), (708, 1121): (128, 128, 128), (1264, 685): (128, 128, 128), (1270, 429): (128, 128, 128), (1481, 691): (52, 194, 95), (1745, 569): (52, 194, 95), (1470, 445): (52, 194, 95), (1494, 843): (128, 128, 128), (1260, 950): (52, 194, 95), (1599, 1493): (128, 128, 128), (1722, 1328): (128, 128, 128), (1582, 1154): (128, 128, 128), (1892, 1770): (52, 194, 95), (1352, 1243): (52, 194, 95), (1639, 1759): (128, 128, 128), (1993, 761): (52, 194, 95), (2358, 898): (128, 128, 128), (2177, 1068): (128, 128, 128), (2363, 549): (128, 128, 128), (2716, 744): (52, 194, 95), (2645, 277): (52, 194, 95), (1833, 996): (128, 128, 128), (2398, 704): (128, 128, 128), (2406, 1142): (128, 128, 128), (2198, 348): (52, 194, 95), (2047, 489): (52, 194, 95), (2420, 273): (128, 128, 128), (2749, 1647): (52, 194, 95), (2464, 1442): (128, 128, 128), (2715, 1396): (52, 194, 95), (2581, 1751): (52, 194, 95)}


In [ ]:
color_territories(clear_image_path, territory_colors, territory_armies, output_path)

Image saved to data/colored_map.png
